# Mount Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!ls "/content/drive/MyDrive/Digital Imaging/final-project/8_move_frames"

img10.jpg  img13.jpg  img16.jpg  img19.jpg  img3.jpg  img6.jpg	img9.jpg
img11.jpg  img14.jpg  img17.jpg  img1.jpg   img4.jpg  img7.jpg
img12.jpg  img15.jpg  img18.jpg  img2.jpg   img5.jpg  img8.jpg


In [3]:
# folder of frames from videoes
folder_path = "/content/drive/MyDrive/Digital Imaging/final-project/8_move_frames"

In [4]:
import shutil

def save_folder_to_drive(colab_folder_path, drive_folder_path, mount_drive = False):
    """
    Saves a folder from the Colab runtime to a directory in Google Drive.

    Args:
        colab_folder_path (str): Path to the folder in the Colab runtime.
        drive_folder_path (str): Path to the destination folder in Google Drive.

    Returns:
        str: Message indicating success or error.
    """
    try:
        if (mount_drive):
            drive.mount('/content/drive')

        # Ensure the Google Drive directory exists
        full_drive_path = os.path.join('/content/drive/My Drive', drive_folder_path)
        if not os.path.exists(full_drive_path):
            os.makedirs(full_drive_path)

        # Copy the folder from Colab runtime to Google Drive
        shutil.copytree(colab_folder_path, os.path.join(full_drive_path, os.path.basename(colab_folder_path)))

        return f"Folder successfully saved to {os.path.join(full_drive_path, os.path.basename(colab_folder_path))}"
    except Exception as e:
        return f"An error occurred: {e}"

# Board Mapping

## Import Libraries

In [13]:
import math
import operator
import sys
from collections import defaultdict

import numpy as np
import cv2

import scipy.spatial as spatial
import scipy.cluster as clstr
import matplotlib.pyplot as plt

import glob
import os

## Functions

In [14]:
def canny(img):
    # Maybe add some auto thresholding here
    edges = cv2.Canny(img, 80, 200)
    return edges


def hough_lines(img):
    rho, theta, thresh = 2, np.pi / 180, 600
    return cv2.HoughLines(img, rho, theta, thresh)


def sort_lines(lines):
    """
    Sorts lines by horizontal and vertical
    """
    h = []
    v = []
    for i in range(lines.shape[0]):
        rho = lines[i][0][0]
        theta = lines[i][0][1]
        if theta < np.pi / 4 or theta > np.pi - np.pi / 4:
            v.append([rho, theta])
        else:
            h.append([rho, theta])
    return h, v


def calculate_intersections(h, v):
    """
    Finds the intersection of two lines given in Hesse normal form.
    See https://stackoverflow.com/a/383527/5087436
    """
    points = []
    for rho1, theta1 in h:
        for rho2, theta2 in v:
            A = np.array([
                [np.cos(theta1), np.sin(theta1)],
                [np.cos(theta2), np.sin(theta2)]
            ])
            b = np.array([[rho1], [rho2]])
            point = np.linalg.solve(A, b)
            point = int(np.round(point[0])), int(np.round(point[1]))
            points.append(point)
    return np.array(points)


def cluster_intersections(points, max_dist=40):
    # I want to change this to kmeans
    Y = spatial.distance.pdist(points)
    Z = clstr.hierarchy.single(Y)
    T = clstr.hierarchy.fcluster(Z, max_dist, 'distance')
    clusters = defaultdict(list)
    for i in range(len(T)):
        clusters[T[i]].append(points[i])
    clusters = clusters.values()
    clusters = map(lambda arr: (np.mean(np.array(arr)[:, 0]), np.mean(np.array(arr)[:, 1])), clusters)

    result = []
    for point in clusters:
        result.append([point[0], point[1]])
    return result


def find_chessboard_corners(points):
    """
    Code from https://medium.com/@neshpatel/solving-sudoku-part-ii-9a7019d196a2
    """
    # Bottom-right point has the largest (x + y) value
    # Top-left has point smallest (x + y) value
    # Bottom-left point has smallest (x - y) value
    # Top-right point has largest (x - y) value
    bottom_right, _ = max(enumerate([pt[0] + pt[1] for pt in points]), key=operator.itemgetter(1))
    top_left, _ = min(enumerate([pt[0] + pt[1] for pt in points]), key=operator.itemgetter(1))
    bottom_left, _ = min(enumerate([pt[0] - pt[1] for pt in points]), key=operator.itemgetter(1))
    top_right, _ = max(enumerate([pt[0] - pt[1] for pt in points]), key=operator.itemgetter(1))
    return [points[top_left], points[top_right], points[bottom_left], points[bottom_right]]


def distance_between(p1, p2):
    """
    Code from https://medium.com/@neshpatel/solving-sudoku-part-ii-9a7019d196a2
    """
    a = p2[0] - p1[0]
    b = p2[1] - p1[1]
    return np.sqrt((a ** 2) + (b ** 2))


def warp_image(img, edges):
    """
    Code from https://medium.com/@neshpatel/solving-sudoku-part-ii-9a7019d196a2
    """
    top_left, top_right, bottom_left, bottom_right = edges[0], edges[1], edges[2], edges[3]

    # Explicitly set the data type to float32 or 'getPerspectiveTransform' will throw an error
    warp_src = np.array([top_left, top_right, bottom_right, bottom_left], dtype='float32')

    side = max([
        distance_between(bottom_right, top_right),
        distance_between(top_left, bottom_left),
        distance_between(bottom_right, bottom_left),
        distance_between(top_left, top_right)
    ])

    # Describe a square with side of the calculated length, this is the new perspective we want to warp to
    warp_dst = np.array([[0, 0], [side - 1, 0], [side - 1, side - 1], [0, side - 1]], dtype='float32')

    # Gets the transformation matrix for skewing the image to fit a square by comparing the 4 before and after points
    m = cv2.getPerspectiveTransform(warp_src, warp_dst)

    # Performs the transformation on the original image
    return cv2.warpPerspective(img, m, (int(side), int(side)))


def cut_chessboard(img, output_path, output_prefix=""):
    side_len = int(img.shape[0] / 8)
    for i in range(8):
        for j in range(8):
            tile = img[i * side_len: (i + 1) * side_len, j * side_len: (j + 1) * side_len]
            cv2.imwrite(output_path + output_prefix + "-" + str(j + i * 8) + ".jpg", tile)


def resize_image(img):
    """
    Resizes image to a maximum width of 800px
    """
    width = img.shape[1]
    if width > 800:
        scale = 800 / width
        return cv2.resize(img, None, fx=scale, fy=scale)
    else:
        return img


def process_chessboard(src_path, output_path, output_prefix="", debug=False, count=0):
    src = cv2.imread(src_path)

    if src is None:
        sys.exit("There is no file with this path!")

    src = resize_image(src)
    src_copy = src.copy()

    # Convert to grayscale
    process = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)

    if debug:
        plt.imshow(process)
        plt.show()

    # Blur to remove disturbing things
    process = cv2.blur(process, (4, 4))

    if debug:
        plt.imshow(process)
        plt.show()

    # Use Canny Edge Detector https://en.wikipedia.org/wiki/Canny_edge_detector
    process = canny(process)

    if debug:
        plt.imshow(process)
        plt.show()

    # Dilate image (thicker lines)
    process = cv2.dilate(process, np.ones((3, 3), dtype=np.uint8))

    if debug:
        plt.imshow(process)
        plt.show()
    # Use Hough transform to detect lines https://en.wikipedia.org/wiki/Hough_transform
    lines = hough_lines(process)

    # Sort lines by horizontal and vertical
    h, v = sort_lines(lines)

    if debug:
        render_lines(src_copy, h, (0, 255, 0))
        render_lines(src_copy, v, (0, 0, 255))
        plt.imshow(process)
        plt.show()

    if len(h) < 9 or len(v) < 9:
        print("There are not enough horizontal and vertical lines in this image. Try it anyway!")

    # Calculate intersections of the horizontal and vertical lines
    intersections = calculate_intersections(h, v)

    if debug:
        render_intersections(src_copy, intersections, (255, 0, 0), 1)
        plt.imshow(process)
        plt.show()

    # Cluster intersection since there are many
    clustered = cluster_intersections(intersections)

    if debug:
        src_copy = src.copy()
        render_intersections(src_copy, clustered, (255, 0, 0), 5)
        plt.imshow(process)
        plt.show()

    if len(clustered) != 81:
        print("Something is wrong. There are " + str(len(intersections)) + " instead of 81 intersections.")

    # Find outer corners of the chessboard
    corners = find_chessboard_corners(clustered)

    if debug:
        src_copy = src.copy()
        render_intersections(src_copy, corners, (255, 0, 0), 5)
        plt.imshow(process)
        plt.show()

    # Warp and crop image
    dst = warp_image(src, corners)
    try:
        cv2.imwrite(output_path + "crop" + str(count) + ".jpg", dst)
    except:
        print("upload fail")
    plt.show()
    if debug:
        plt.imshow(process)
        plt.show()

def render_lines(img, lines, color):
    for rho, theta in lines:
        a = math.cos(theta)
        b = math.sin(theta)
        x0, y0 = a * rho, b * rho
        pt1 = (int(x0 + 1000 * (-b)), int(y0 + 1000 * a))
        pt2 = (int(x0 - 1000 * (-b)), int(y0 - 1000 * a))
        cv2.line(img, pt1, pt2, color, 1, cv2.LINE_AA)


def render_intersections(img, points, color, size):
    for point in points:
        cv2.circle(img, (int(point[0]), int(point[1])), 2, color, size)

In [15]:
def line_intersections(h_lines, v_lines):
  points = []
  for x,[r_h, t_h] in h_lines:
    row = []
    for y,[r_v, t_v] in v_lines:
      a = np.array([[np.cos(t_h), np.sin(t_h)], [np.cos(t_v), np.sin(t_v)]])
      b = np.array([r_h, r_v])
      inter_point = np.linalg.solve(a, b)
      row.append((int(inter_point[0]),int(inter_point[1])))
    points.append(row)
  return np.array(points)

In [16]:
def plot_grid(image, verbose = False):
  def get_lines(lines,length,isHorizon):
    tmp,res, =[],[]
    for rho, theta in lines:
        a = math.cos(theta)
        b = math.sin(theta)
        x0, y0 = a * rho, b * rho
        pts = [int(x0 + 1000 * (-b)), int(y0 + 1000 * a) ,int(x0 - 1000 * (-b)), int(y0 - 1000 * a)]
        if(isHorizon):
          tmp.append([max(pts[1],pts[3]),[rho,theta]])
        else:
          tmp.append([max(pts[0],pts[2]),[rho,theta]])
    tmp.sort()
    mean = length/8
    cur = tmp[0]
    for i in range(len(tmp)):
      if(abs(abs(tmp[i][0]-cur[0])-mean)<=25):
        res.append(cur)
        cur = tmp[i]
      if(i==len(tmp)-1 and len(res)<9):
        res.append(cur)
    return res

  def locate_board(h,v,r_size,c_size):
    start_point = (v[0],h[0])
    row_length = v[-1]-v[0]
    col_length = h[-1]-h[0]
    return start_point,row_length,col_length



  img = image.copy()
  img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  img = canny(img)
  img = cv2.dilate(img, np.ones((3, 3), dtype=np.uint8),3)
  lines = hough_lines(img)
  h, v = sort_lines(lines)
  h = get_lines(h,image.shape[0],True)
  v = get_lines(v,image.shape[1],False)
  points = line_intersections(h,v)

  table = []
  for i in range(8):
    rr = []
    for j in range(8):
        rr.append([points[i][j][0],points[i][j][1],points[i+1][j+1][0],points[i+1][j+1][1]])
    #   rr.append([points[j][i][0],points[j][i][1],points[j+1][i+1][0],points[j+1][i+1][1]])
    table.append(rr)


  for r in table:
    for pts in r:
      cv2.rectangle(image, (pts[0], pts[1]), (pts[2], pts[3]), (0, 255, 0), 2)

  if (verbose):
    plt.imshow(image)
    plt.show()

  return table

## Map board

In [17]:
!rm -rf "/content/output"
!mkdir "/content/output"

In [18]:
output_path = 'output/'

filenames = os.listdir(folder_path)
data = []

for filename in filenames:
    frame_id = int(filename.split(".")[0].strip("img"))
    data.append((frame_id, filename))
data.sort()


for (cnt, path) in data:
    full_path = os.path.join(folder_path, path)
    process_chessboard(full_path, "output/", "", False, cnt)

<ipython-input-14-10b2beae692d>:42: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  point = int(np.round(point[0])), int(np.round(point[1]))


Something is wrong. There are 224 instead of 81 intersections.
Something is wrong. There are 342 instead of 81 intersections.
Something is wrong. There are 238 instead of 81 intersections.
Something is wrong. There are 221 instead of 81 intersections.
Something is wrong. There are 266 instead of 81 intersections.
Something is wrong. There are 324 instead of 81 intersections.
Something is wrong. There are 272 instead of 81 intersections.
Something is wrong. There are 323 instead of 81 intersections.
Something is wrong. There are 342 instead of 81 intersections.
Something is wrong. There are 340 instead of 81 intersections.
Something is wrong. There are 306 instead of 81 intersections.
Something is wrong. There are 252 instead of 81 intersections.
Something is wrong. There are 270 instead of 81 intersections.
Something is wrong. There are 270 instead of 81 intersections.
Something is wrong. There are 306 instead of 81 intersections.
Something is wrong. There are 225 instead of 81 interse

In [34]:
cropped_filenames = os.listdir("output/")
cropped_filenames.sort(key=lambda s: int(s.split(".")[0][4:]))
frames_corners = []

for path in cropped_filenames:
    img = cv2.imread(os.path.join("output/", path))
    tb = plot_grid(img, verbose = False)
    frames_corners.append(np.array(tb))

In [20]:
assert len(frames_corners) == len(os.listdir(folder_path))

# YOLO setup

## Ultralytics setup

In [5]:
!pip install ultralytics==8.3.40

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 898.5/898.5 kB 28.8 MB/s eta 0:00:00


In [6]:
import ultralytics
ultralytics.checks()

Ultralytics 8.3.40 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 32.6/235.7 GB disk)


In [7]:
from ultralytics import YOLO
from ultralytics import settings

In [11]:
model = YOLO("yolov8n.pt")

# Detect pieces Train

## Roboflow Setup

In [16]:
!pip install roboflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.9/80.9 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 6.2 MB/s eta 0:00:00
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10


## Import Libraries

In [17]:
from IPython.display import display, Image
from roboflow import Roboflow

## Load data

In [ ]:
API_KEY = input("Enter Roboflow API key: ")

In [19]:
rf = Roboflow(api_key=API_KEY)
project = rf.workspace("digimageproject").project("chess-y67ho")
version = project.version(1)
dataset = version.download("yolov8")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Chess-1 in yolov8:: 100%|██████████| 990/990 [00:00<00:00, 2746.10it/s]


## Train

Load pretrained YOLO

In [23]:
model = YOLO('yolo8n.pt')

In [25]:
train_result = model.train(data='/content/Chess-1/data.yaml', epochs=60, batch=64)  # train the model

New https://pypi.org/project/ultralytics/8.3.48 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.40 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolo8n.pt, data=/content/Chess-1/data.yaml, epochs=60, time=None, patience=100, batch=64, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train32, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames

train: Scanning /content/Chess-1/train/labels.cache... 391 images, 0 backgrounds, 0 corrupt: 100%|██████████| 391/391 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning /content/Chess-1/valid/labels.cache... 49 images, 0 backgrounds, 0 corrupt: 100%|██████████| 49/49 [00:00<?, ?it/s]


Plotting labels to runs/detect/train32/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000625, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to runs/detect/train32
Starting training for 60 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

      60/60      8.52G     0.5127     0.5082     0.8426          8        640: 100%|██████████| 7/7 [00:04<00:00,  1.54it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  1.66it/s]

                   all         49        216      0.987      0.985      0.994      0.873



60 epochs completed in 0.145 hours.
Optimizer stripped from runs/detect/train32/weights/last.pt, 6.3MB
Optimizer stripped from runs/detect/train32/weights/best.pt, 6.3MB

Validating runs/detect/train32/weights/best.pt...
Ultralytics 8.3.40 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3,007,988 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  2.54it/s]


                   all         49        216       0.97      0.991      0.994      0.876
                     0          9         13      0.957          1      0.995       0.88
                     1          8         26      0.992          1      0.995      0.859
                    10         10         15          1      0.916      0.995      0.852
                    11          9          9      0.937          1      0.995       0.88
                     2          9         16          1      0.986      0.995      0.841
                     3          8         14      0.942          1      0.995      0.887
                     4         12         18      0.899          1      0.987      0.856
                     5         15         22      0.982          1      0.995      0.928
                     6         13         43          1      0.993      0.995      0.874
                     7         11         11       0.96          1      0.995      0.889
                     

In [29]:
save_res_path = input("Enter folder name for saved logs: ")
save_folder_to_drive("/content/runs/detect", f"/content/drive/MyDrive/Digital Imaging/result/{save_res_path}")

Enter folder name for saved logs: 9-dec


'Folder successfully saved to /content/drive/MyDrive/Digital Imaging/result/9-dec/detect'

## Test

# Detect Piece Inference

In [189]:
results = []
frames_classes = []
frames_coors = []
frames_confs = []

for path in cropped_filenames:
    img = cv2.imread(os.path.join("output/", path))
    res = model(img, verbose = False, conf = 0)[0]
    classes_labels = np.array(res.boxes.cls.to("cpu"), dtype=int)
    locs = res.boxes.xyxy.to("cpu").numpy()
    confs = res.boxes.conf.to("cpu").numpy()

    results.append(res.to("cpu"))
    frames_classes.append(classes_labels)
    frames_coors.append(locs)
    frames_confs.append(confs)

## Map infereneced result to coordinate

In [93]:
assert len(frames_classes) == len(frames_corners)
assert len(frames_coors) == len(frames_corners)
assert len(frames_confs) == len(frames_corners)

In [175]:
label_to_piece_names = [
    "white-queen",
    "white-pawn",
    "black-rook",
    "black-bishop",
    "black-knight",
    "black-queen",
    "black-pawn",
    "black-king",
    "white-rook",
    "white-bishop",
    "white-knight",
    "white-king"
]

columns = ["h", "g", "f", "e", "d", "c", "b", "a"]
rows = [i+1 for i in range(8)]

In [206]:
encoded_frames = []

for i in range(len(frames_corners)): # for each frame

    board = [["None" for _ in range(8)] for _ in range(8)]
    piece_conf = [[None for _ in range(8)] for _ in range(8)]

    corners = frames_corners[i]
    classes = frames_classes[i]
    locs = frames_coors[i]
    confs = frames_confs[i]
    filename = cropped_filenames[i]
    # board_bgr = cv2.imread(f"output/{filename}")
    # board = cv2.cvtColor(board_bgr, cv2.COLOR_BGR2RGB)

    for (j, (cls, (x1, y1, x2, y2), conf)) in enumerate(zip(classes, locs, confs)): # for each piece found
        x1, y1, x2, y2 = round(x1), round(y1), round(x2), round(y2)

        x_mid = (x1 + x2) // 2
        y_base = y2 - (y2 - y1) // 10
        located = False
        # print(f"(x,y) = ({x_mid}, {y_base})")

        for row in range(8):
            if (located):
                break
            for col in range(8):
                if (located):
                    break
                x_l, y_t, x_r, y_b = corners[row][col]

                assert(x_l < x_r)
                assert(y_t < y_b)

                if ((x_l <= x_mid <= x_r) and (y_t <= y_base <= y_b)):
                    # print(f"Found {label_to_piece_names[cls]} at {columns[col]}{rows[row]} with conf: {conf:.4f}")
                    if (piece_conf[row][col] is None and conf > 0.6):
                        piece_conf[row][col] = conf
                        board[row][col] = label_to_piece_names[int(results[0].names[cls])]
                        located = True

    encoded_frames.append(board)

# Decode board to PGN